In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

import seaborn as sns



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))


# Any results you write to the current directory are saved as output.

In this kernell, we will look the some important datas of McDonalds menu.

Firstly we start with recoginizing the data.


In [ ]:
data = pd.read_csv("../input/menu.csv")

In [ ]:
data.head(10)

In [ ]:
data.info()

In the upper table, we figure out that all features have 260 elements which means we don't have missing values, but to be sure we can see the code below.

In [ ]:
  for each in data.columns:
    assert  data[each].notnull().all() # returns nothing so there is no missing value

Now, we will see the menu diversity in the pie chart.

In [ ]:
category_data = data["Category"]
labels = category_data.value_counts().index
fig = {
    "data" : [
        { "values" : category_data.value_counts(),
        "labels" : labels,
        "domain": {"x": [0, .5]},
        "name": "Number Of production diversity",
        "type" : "pie"
        },
    ],
    "layout" : {
        "title":"McDonalds food types",
        "annotations": [
            { "font": { "size": 20},
              "showarrow": False,
              "text": "Ratio of Categories",
                "x": 0.20,
                "y": 1
            },
        ]
    } 
    
}
iplot(fig)


So in the upper pie chart, we realize that McDonalds have lots of Coffe and Tea option.

In [ ]:

breakfast_data = data[data.Category == "Breakfast" ]
breakfast_data.plot(kind = "hist",y = "Total Fat",bins = 50,range= (0,250),normed = True)

In [ ]:
trace = go.Bar(
x =breakfast_data.Item,
y = breakfast_data["Total Fat"],
    name = "Total Fat graph",
    marker = dict(color = "rgba(255, 174, 255, 0.5)"),
    )

data = [trace]

layout = go.Layout(barmode = "group")

fig = go.Figure(data = data, layout = layout)
iplot(fig)
    

In [ ]:
# We change the Serving Size feature of breakfast data
# Just leave gram values on the data

gram_list = []
for each in breakfast_data["Serving Size"]:
    gram_list.append(each.split()[2])

temp_list =[]
new_list = []
for i in gram_list:
    temp_list = list(i)
    temp_list.remove("(")
    a = "".join(temp_list)
    new_list.append(a)
    temp_list =[]
    

gram_values = [] 
for i in new_list:
    gram_values.append(int(i))

#gram_values
breakfast_data["Serving Size"] = gram_values    
#breakfast_data
#breakfast_data["Serving Size"]
#temp_list = list(each)

g = sns.jointplot(breakfast_data["Serving Size"], breakfast_data["Cholesterol"]
                  , kind="kde", size=7)
plt.savefig('graph.png')
plt.show()
    
    
    

In the below you can find a heat map which shows the correlation between all the features.

In [ ]:

f,ax = plt.subplots(figsize=(12, 12))
sns.heatmap(breakfast_data.corr(),
            annot=True, linewidths=0.5,linecolor="red", fmt= '.1f',ax=ax)
plt.show()

We can find meals with the most calorie

In [ ]:
data = pd.read_csv("../input/menu.csv")
filtre_Cal = data.Calories > 800

filtred_data = data[filtre_Cal]
filtred_data

plt.scatter(filtred_data["Calories"],filtred_data.Item, color ="red", label="Calories")

In [ ]:
protein_color = [float(each) for each in data["Protein"]]
data = [
    {
        'x': data["Carbohydrates (% Daily Value)"],
        'y': data["Iron (% Daily Value)"],
        'mode': 'markers',
        'marker': {
            'color': protein_color,
            'size': data["Calcium (% Daily Value)"],
            'showscale': True
        },
        "text" :  data.Item  
    }
]
iplot(data)

In the upper graph shows us the protein, Carbohydrates, iron,calcium datas together

Lastly, we can see the Vitamin A and vitamin C datas of top three meal with high fat

In [ ]:

data = pd.read_csv("../input/menu.csv")
filter2 = data["Total Fat"] > 50
new_data = data[filter2]
new_data

In [ ]:



fat_list = data["Total Fat"]

fat_list = list(fat_list)

a =0
count =0
top_list = []
while count < 3:
    for each in fat_list:
        if each > a:
            a = each
    top_list.append(a)
    fat_list.remove(a)
    a=0
    count += 1
#top_list

count = 0
name_list = []
for num in top_list:
    for each in data.Item:
        if data["Total Fat"][count] == num:
            name_list.append(each)
        count += 1
    count =0


    
count = 0
vitamin_A_list = []
for num in top_list:
    for each in data["Vitamin A (% Daily Value)"]:
        if data["Total Fat"][count] == num:
            vitamin_A_list.append(each)
        count += 1
    count = 0

   

    
count = 0
vitamin_C_list = []
for num in top_list:
    for each in data["Vitamin C (% Daily Value)"]:
        if data["Total Fat"][count] == num:
            vitamin_C_list.append(each)
        count += 1
    count = 0
    
vitamin_C_list

name_list = pd.DataFrame(name_list)
vitamin_A_list= pd.DataFrame(vitamin_A_list)
vitamin_C_list = pd.DataFrame(vitamin_C_list)
top_list = pd.DataFrame(top_list)

top_three_data = pd.concat([name_list,vitamin_A_list,
                            vitamin_C_list,top_list ], axis =1)

top_three_data


We can get the top three fat value.

In [ ]:

trace1 = {
  'x': top_three_data.iloc[:,0],
  'y': top_three_data.iloc[:,1],
  'name': 'vitamin A',
  'type' : "bar"
};

trace2 = {
  'x': top_three_data.iloc[:,0],
  'y': top_three_data.iloc[:,2],
  'name': 'vitamin C',
  'type': 'bar'
};

data = [trace1 , trace2]

layout = {
  'xaxis': {'title': 'Top 3 meal with total fat'},
  'barmode': 'relative',
  'title': 'Vitamin A and C values of 3 meal which has the most fat value'
};

fig = go.Figure(data = data, layout = layout)
iplot(fig)

So we can realize that, chicken mcNuggets have no vitamin A but breakfast has both of the vitamins.